### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

mkdir: cannot create directory ‘/root/airflow/dags’: File exists


In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [ ]:
# Запуск шедулера
!airflow scheduler -D

/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'Serialize

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken  # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http 18273 > /dev/null &

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: redirecting stderr to stdout


### Задача на разработку

Вам дан код который возвращает в Xcom значение от 1 до 10. Ваша задача реализовать BranchOperator который в зависимости от этого значения вернет одну из задач, больше или меньше. Вам дан кусочек кода который необходимо доработать.


```python
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
from airflow.operators.dummy import DummyOperator
from airflow.operators.python_operator import BranchPythonOperator
from random import randint

default_args = {
    'owner':'airflow',
    'start_date': datetime(2022, 2, 16)
}

dag = DAG('dag', schedule_interval='@daily', default_args=default_args)

def rand(**kwargs):
  kwargs['ti'].xcom_push(key='rand', value=randint(0, 10))


def branch(**kwargs):
# ВАШ КОД
# Если > 5 то вернуть higher 
# иначе вернуть lower

lower = DummyOperator(
    task_id = 'lower',
    dag=dag
)

higher = DummyOperator(
    task_id = 'higher',
    dag=dag
)

branch_op = BranchPythonOperator(
    task_id = 'branch_task',
    provide_context = True,
    python_callable=branch,
    dag=dag
)

random_number = PythonOperator(
    task_id = 'random_number',
    python_callable=rand,
    dag=dag
)

random_number >> branch_op >> [lower, higher]
```

In [ ]:
## ВАШЕ РЕШЕНИЕ

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
from airflow.operators.dummy import DummyOperator
from airflow.operators.python_operator import BranchPythonOperator
from random import randint

default_args = {
    'owner':'airflow',
    'start_date': datetime(2022, 2, 16)
}

dag = DAG('dag', schedule_interval='@daily', default_args=default_args)

def rand(**kwargs):
  kwargs['ti'].xcom_push(key='rand', value=randint(0, 10))


def branch(**kwargs):
    xcom_value = int(kwargs['ti'].xcom_pull(task_ids='random_number', key='rand'))
    if xcom_value >= 5:
        return 'higher'
    else:
        return 'lower'

lower = DummyOperator(
    task_id = 'lower',
    dag=dag
)

higher = DummyOperator(
    task_id = 'higher',
    dag=dag
)

branch_op = BranchPythonOperator(
    task_id = 'branch_task',
    provide_context = True,
    python_callable=branch,
    dag=dag
)

random_number = PythonOperator(
    task_id = 'random_number',
    python_callable=rand,
    dag=dag
)

random_number >> branch_op >> [lower, higher]

[<Task(DummyOperator): lower>, <Task(DummyOperator): higher>]